In [17]:
import numpy as np
from numpy import sqrt,exp,cos,sin,transpose,inf
from numpy.linalg import eig
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp,complex_ode,odeint
import sympy
from sympy import MatrixSymbol,MatAdd,MatMul,Identity,I,Matrix,symbols
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.utilities.iterables import flatten
import time
from IPython.core.display import display,HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

from rbconsts import *
from physconsts import *

In [18]:
def odeintz(func, z0, t,**kwargs):
    """An odeint-like function for complex valued differential equations."""

    # Disallow Jacobian-related arguments.
    _unsupported_odeint_args = ['Dfun', 'col_deriv', 'ml', 'mu']
    bad_args = [arg for arg in kwargs if arg in _unsupported_odeint_args]
    if len(bad_args) > 0:
        raise ValueError("The odeint argument %r is not supported by "
                         "odeintz." % (bad_args[0],))

    # Make sure z0 is a numpy array of type np.complex128.
    z0 = np.array(z0, dtype=np.complex128, ndmin=1)

    def realfunc(x, t, *args):
        z = x.view(np.complex128)
        dzdt = func(z, t, *args)
        # func might return a python list, so convert its return
        # value to an array with type np.complex128, and then return
        # a np.float64 view of that array.
        return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

#     def realfunc(x, t, params):
#         z = x.view(np.complex128)
#         dzdt = func(z, t, params)
#         # func might return a python list, so convert its return
#         # value to an array with type np.complex128, and then return
#         # a np.float64 view of that array.
#         return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

    stopwatch = time.time() # time the computation 
    result = odeint(realfunc, z0.view(np.float64), t, **kwargs)
    stopwatch = time.time() - stopwatch

    if kwargs.get('full_output', False):
        z = result[0].view(np.complex128)
        infodict = result[1]
        return z, infodict#, stopwatch
    else:
        z = result.view(np.complex128)
        return z,stopwatch


def cc(z): 
    return np.conj(z)

dt = 0.05 # the timestep for numerical solutions

In [27]:
### EVERYTHING IN HERE IS TEST CODE THAT IS INTENDED TO REPRODUCE THE HARD-
### CODED VON NEUMANN EQS FROM A SYMBOLIC HAMILTONIAN; 

###############################################################################
## ***** SYMBOLIC SYSTEM SOLVER PROTOTYPE *****
## Hamiltonian is wrong but the proof of concept works
## Rabi oscillations with no spontaneous emission, for various values
## of detuning from the transition resonance
###############################################################################

# # initial conditions - system starts in ground state
# rho_ee = 0 + 0j
# rho_gg = 1 + 0j
# rho_eg = 0 + 0j # no initial mixing; this is actually rho^tilda_ge
# y0 = np.array([rho_gg,rho_ee,rho_eg]) # bundle i.c.

### EVERYTHING IN HERE IS TEST CODE THAT IS INTENDED TO REPRODUCE THE HARD-
### CODED VON NEUMANN EQS FROM A SYMBOLIC HAMILTONIAN

def comm(A,B):
    """[A,B]=A.B-B.A. Assumes symbolic matrix."""
    return (MatMul(A,B)-MatMul(B,A)) #.as_mutable

# define symbolic matrices which can be made numeric later
E1,E2,d,O,t = symbols('E1 E2 d O t') # symbolic variables

# Set the parameters of H; arbitrary for now
O = 1; E2 = 1; E1 = 0; d = 1;

H_a = Matrix([[E1,0],[0,E2]])
H_field = Matrix([[0,-cc(O)/2],[-O/2,0]])
H = H_a + H_field # the full Hamiltonian
r = MatrixSymbol('r',2,2).as_mutable() # density op
r[0,1]=cc(r[1,0]) # now we need only solve 3 eqs
RHS = 1j*comm(r,H) # RHS of von Neumann eq system
# rhs = np.array([x for x in  RHS]) # the rhs expressions of the von Neumann eqs
rhs = [x for x in  RHS]

# initial conditions - system starts in ground state
r0 = [1+0j,0+0j,0+0j,0+0j] # flattened initial dens. op.

# generate this derivs function from symbolic Hamiltonian and Lambdify
# TODO: include parameters for H as a tuple
derivs = lambdify((flatten([flatten(x) for x in RHS.free_symbols]),t),rhs)

In [60]:
flatten([flatten(x) for x in RHS.free_symbols])

[r[0, 0], r[0, 1], r[1, 0], r[1, 1]]

In [36]:
rhs

[1.0*I*(-0.5*conjugate(r[1, 0]) + 0.5*r[1, 0]),
 1.0*I*(conjugate(r[1, 0]) - 0.5*r[0, 0] + 0.5*r[1, 1]),
 1.0*I*(0.5*r[0, 0] - r[1, 0] - 0.5*r[1, 1]),
 1.0*I*(0.5*conjugate(r[1, 0]) - 0.5*r[1, 0])]

In [ ]:
t_exp = 10 # experiment duration
ts = np.arange(t_exp,step=dt)

fig = plt.figure()
ax = fig.add_subplot(111)
# ax.set_ylim((0,1))
ax.set_xlim((0,t_exp))
ax.set_title('Rabi oscillations w/o spontaneous emission')
ax.set_xlabel('time [gamma]')
ax.set_ylabel('Excitation probability')

# this doesn't throw any errors, but the result is wrong. 
soln,ctime = odeintz(derivs, r0, ts) #, args=(t1,t2,D,Omega))
ax.plot(ts,soln[:,1]) # plot rho_ee
plt.show()